Please see my modelling notebook. This is only used for the hyperparameter tuning.

In [ ]:
import string
import numpy as np
import pandas as pd
from time import time
from collections import Counter
import torch
import torch.nn as nn
#import nltk
import random
import sys

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
print(torch.__version__)

1.8.1+cpu


In [ ]:
def read_file(filename):
  with open('ptb.train.txt', encoding = 'utf-8') as f:
      raw_text = f.read().replace("\n", "<eos>")
      return raw_text

In [ ]:
with open('ptb.train.txt', encoding = 'utf-8') as f:
      raw_text = f.read()[:100].replace("\n", "<eos>")
raw_text

' aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote'

In [ ]:
with open('ptb.valid.txt', encoding = 'utf-8') as f:
      raw_text = f.read()[:100].replace("\n", "<eos>")
raw_text

' consumers may want to move their telephones a little closer to the tv set <eos> <unk> <unk> watching ab'

In [ ]:
def read_file(filename):
  with open(filename) as f:
      raw_text = f.read().replace("\n", "<eos>").split()
      return raw_text

train_data=read_file('ptb.train.txt')

In [ ]:
train_data[:20]

['aer',
 'banknote',
 'berlitz',
 'calloway',
 'centrust',
 'cluett',
 'fromstein',
 'gitano',
 'guterman',
 'hydro-quebec',
 'ipo',
 'kia',
 'memotec',
 'mlx',
 'nahb',
 'punts',
 'rake',
 'regatta',
 'rubens',
 'sim']

In [ ]:
import torchtext
from torchtext.legacy import data


In [ ]:
def get_data(batch_size=60,bptt_len=22):
  TEXT = data.Field(lower=True,batch_first=True,eos_token=True,unk_token=True,pad_token=True,pad_first=True)
  train, val, test = torchtext.legacy.datasets.LanguageModelingDataset.splits(path=".", 
    train="ptb.train.txt", 
    validation="ptb.valid.txt", 
    test="ptb.test.txt", 
    text_field=TEXT)
  TEXT.build_vocab(train, max_size=10000)
  print("vocabulary size: {}".format(len(TEXT.vocab)))

  VOCAB_SIZE = len(TEXT.vocab)
  train_iter, val_iter, test_iter = data.BPTTIterator.splits(
    (train, val, test), batch_size=batch_size, bptt_len=bptt_len,device=device,repeat=False, shuffle=True)
  return train_iter, val_iter, test_iter
  

In [ ]:
train_iter, val_iter, test_iter=get_data(batch_size=60,bptt_len=20)

vocabulary size: 10001


In [ ]:
it_test = iter(train_iter)
batch = next(it_test)

In [ ]:
batch


[torchtext.legacy.data.batch.Batch of size 60]
	[.text]:[torch.cuda.LongTensor of size 60x20 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 60x20 (GPU 0)]

In [ ]:
batch.text[0]

tensor([9971, 9972, 9973, 9975, 9976, 9977, 9981, 9982, 9983, 9984, 9985, 9987,
        9988, 9989, 9990, 9992, 9993, 9994, 9995, 9996], device='cuda:0')

In [ ]:
batch.target[0]

tensor([9972, 9973, 9975, 9976, 9977, 9981, 9982, 9983, 9984, 9985, 9987, 9988,
        9989, 9990, 9992, 9993, 9994, 9995, 9996, 9997], device='cuda:0')

In [ ]:
TEXT = data.Field(lower=True,batch_first=True,eos_token=True,unk_token=True,pad_token=True,pad_first=True)
train, val, test = torchtext.legacy.datasets.LanguageModelingDataset.splits(path=".", 
    train="ptb.train.txt", 
    validation="ptb.valid.txt", 
    test="ptb.test.txt", 
    text_field=TEXT)
TEXT.build_vocab(train, max_size=10000)
print("vocabulary size: {}".format(len(TEXT.vocab)))

vocabulary size: 10001


In [ ]:
TEXT.vocab.itos[:10]

[True, 'the', '<unk>', '<eos>', 'n', 'of', 'to', 'a', 'in', 'and']

In [ ]:
it_test = iter(train_iter)
batch = next(it_test)
print(" ".join([TEXT.vocab.itos[i] for i in batch.text[0].data])) # where 0 refers to the nth batch
print()
print(" ".join([TEXT.vocab.itos[i] for i in batch.target[0].data]))


aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim

banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food


In [ ]:
batch.target.shape

torch.Size([60, 20])

In [ ]:
from torch.autograd import Variable
class WordLSTM(nn.Module):
    
    def __init__(self,n_hidden=300,vocab_size=10001, n_layers=2, drop_prob=0.1):
        super().__init__()
        self.dropout = nn.Dropout(drop_prob)
        self.emb_layer = nn.Embedding(vocab_size,250)
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        #self.lr = lr
        ## define the LSTM
        self.lstm = nn.LSTM(250, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        ## define a dropout layer
        ## define the fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)
        #self.softmax = nn.LogSoftmax(dim=-1)

        self.init_weights()
        
    def init_weights(self):
        initrange = 0.05
        self.emb_layer.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-initrange, initrange) 

    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        ## pass input through embedding layer
        #print(x.shape)

        embedded = self.emb_layer(x)
        embedded=self.dropout(embedded)
        #hidden = self.init_hidden(bs)
        ## Get the outputs and the new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        ## pass through a dropout layer
        out = self.dropout(lstm_output)
        #print(out.shape)
        #out = out.contiguous().view(-1, self.n_hidden) 

        out = out.reshape(-1, self.n_hidden) 
        #print(out.shape) [1200,300]
        ## put "out" through the fully-connected layer
        out = self.fc(out)
        #print(out.shape) [1200,10001]
        #out = self.softmax(out)
        return out, hidden
      

    def init_hidden(self, batch_size):
        ''' initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))        
      
        return hidden

def repackage_hidden(h):
  """Wraps hidden states in new Variables, to detach them from their history."""
  if isinstance(h, Variable):
    return Variable(h.data)
  else:
    return tuple(repackage_hidden(v) for v in h)

In [ ]:
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if torch.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [ ]:
import torch.nn.functional as F
def idtowordbatch(listofid):
  sent=[]
  for i in range(19):
      a=id_to_word[listofid[i]]
      sent.append(a)
  return sent

In [ ]:
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.exp(cross_entropy)
    return perplexity

In [ ]:
def train(net, data, optimizer ,batch_size=60,clip=0.25,num_steps=20,print_every=256):
    # optimize
    
    # loss
    criterion = nn.CrossEntropyLoss()

    
    # push model to GPU
    net.cuda()
    
    counter = 0
    cumulative_loss = 0.
    cumulative_perp=0.
    loss = 0
    train_loss=0
    iters =0
    net.train()

    # initialize hidden state
    h = net.init_hidden(batch_size)
        
    for i, batch in enumerate(data):
        inputs, targets = batch.text, batch.target
        counter+= 1
        inputs = inputs.to(dtype=torch.long)
        targets = targets.to(dtype=torch.long)    
              # push tensors to GPU
        inputs, targets = inputs.cuda(), targets.cuda()

            # detach hidden states
        h = tuple([each.data for each in h])

            # zero accumulated gradients
        net.zero_grad()
            
            # get the output from the model
        output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
        loss = criterion(output, targets.view(-1))
            # back-propagate error
        loss.backward()
        train_loss+= loss.item()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
            # update weigths
        optimizer.step()
            # Resets the gradients
        optimizer.zero_grad()  
    epoch_train_loss =train_loss / len(data)
    train_ppl = np.exp(epoch_train_loss)
              
    return epoch_train_loss, train_ppl

In [ ]:
def test(net,data, batch_size=60, clip=0.25,model_name="lm_model"):
    # loss
    criterion = nn.CrossEntropyLoss()
    es = EarlyStopping(patience=5)
    
    # push model to GPU
    net.cuda()    
    counter = 0
    iters=0
    valid_loss = 0.
    best=np.inf
    net.eval()
    loss = 0
    h = net.init_hidden(batch_size)
    with torch.no_grad():    
      for i, batch in enumerate(data):
          inputs, targets = batch.text, batch.target
          counter+= 1
          inputs = inputs.to(dtype=torch.long)
          targets = targets.to(dtype=torch.long)    
              # push tensors to GPU
          inputs, targets = inputs.cuda(), targets.cuda()
              
              # detach hidden states
          h = tuple([each.data for each in h])
          #h = repackage_hidden(h)    
              # get the output from the model
          output, h = net(inputs, h)
              
          # calculate the loss and perform backprop
          loss = criterion(output, targets.view(-1))
          valid_loss += loss.item()
          
          
      epoch_valid_loss = valid_loss / len(data)
      valid_ppl = np.exp(epoch_valid_loss)
      if valid_loss < best:
          best = valid_loss
          torch.save(net, model_name+".pt")
          print("model saved.")
      
    
    
    return epoch_valid_loss, valid_ppl                


In [ ]:
def find_class(label):
  classs=[]
  for i in range(len(label)):
    a=label[i].tolist()
    classs.append(a.index(max(a)))
  return classs


In [ ]:
batch.text.shape

torch.Size([60, 20])

In [ ]:
from itertools import product
def log_values(writer, step, loss, perplexity, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/perplexity", perplexity, step)

# Hyperparameters
parameters = dict(
    lr = [0.001,0.005,0.01]
    bptt_len = [20 30,40]
    n_hidden = [300,350,400]
    batch_size = [30,45,60]
)
param_values = [v for v in parameters.values()]

from torch.utils.tensorboard import SummaryWriter
for run_id, (lr,bptt_len, n_hidden,batch_size) in enumerate(product(*param_values)):
  # Creates a logger for the experiment
  
  comment = f' bptt_len = {bptt_len} lr = {lr} n_hidden = {n_hidden} batch_size = {batch_size}'
  writer = SummaryWriter(comment=comment)
  #Get Data
  train_iter, val_iter, test_iter=get_data(batch_size=batch_size,bptt_len=bptt_len)
  print(train_iter)
  it_test = iter(train_iter)
  batch = next(it_test)
  # instantiate the model
  net = WordLSTM(n_hidden=n_hidden,vocab_size=10002, n_layers=2, drop_prob=0.6)
  h = net.init_hidden(batch_size)
  #h=torch.tensor(h)
  net.cuda()
  print(net)
  print(comment)
  optimizer = torch.optim.RMSprop(net.parameters(), lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
  opt = torch.optim.Adam(net.parameters(), lr = lr)
  es = EarlyStopping(patience=5)
  for e in range(2):

    train_loss, train_perplexity = train(net,train_iter,optimizer=optimizer,batch_size = batch_size)
    val_loss, valid_perplexity = test(net,val_iter,batch_size =batch_size,model_name="lm_model"+str(run_id))
    # Logs to TensorBoard
    if es.step(torch.from_numpy(np.array([val_loss]))):
          break
    log_values(writer, e, train_loss, train_perplexity, "Train")
    log_values(writer, e, val_loss, valid_perplexity, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training perplexity {:.2f}'.format(train_loss, train_perplexity))
    print('\t Validation loss {:.5f}, Validation perplexity {:.2f}'.format(val_loss, valid_perplexity))
    print('-----------------------------------------------------')

  # Compute final evaluation results
  print('After training:')
  #train_loss, train_perplexity = test(net,train_iter ,batch_size = batch_size)
  val_loss, valid_perplexity = test(net,val_iter ,batch_size = batch_size)
  test_loss, test_perplexity = test(net,test_iter ,batch_size = batch_size)

  # Logs to TensorBoard
  log_values(writer, e+1, train_loss, train_perplexity, "Train")
  log_values(writer, e+1, val_loss, valid_perplexity, "Validation")
  #log_values(writer, e+1, test_loss, test_perplexity, "Test")

  #print('\t Training loss {:.5f}, Training perplexity {:.2f}'.format(train_loss, train_perplexity))
  print('\t Validation loss {:.5f}, Validation perplexity {:.2f}'.format(val_loss, valid_perplexity))
  print('\t Test loss {:.5f}, Test perplexity {:.2f}'.format(test_loss, test_perplexity))
  print('-----------------------------------------------------')

  writer.add_hparams(
            {"lr": lr, "bptt_len": bptt_len, "n_hidden":n_hidden,"batch_size":batch_size},
            {
                "test_perplexity": test_perplexity,
                "test_loss": test_loss,
            }
        )
  inputs = iter(train_iter)
  inputs=next(inputs)
  net.lstm.all_weights
  writer.add_graph(net,(inputs.text,h),verbose=True)
  writer.add_histogram('lstm.hidden-hidden.weight0', net.lstm.weight_hh_l0, e)
  writer.add_histogram('conv1.hidden-hidden.weight1', net.lstm.weight_hh_l1, e)
  writer.add_histogram('conv1.input-hidden.weight0', net.lstm.weight_ih_l0, e)
  writer.add_histogram('conv1.input-hidden.weight1', net.lstm.weight_ih_l1, e)
  writer.add_histogram('conv1.hidden-hidden.bias0', net.lstm.bias_hh_l0, e)
  writer.add_histogram('conv1.hidden-hidden.bias1', net.lstm.bias_hh_l1, e)
  writer.add_histogram('conv1.input-hidden.bias0', net.lstm.bias_ih_l0, e)
  writer.add_histogram('conv1.input-hidden.bias1', net.lstm.bias_ih_l1, e)

  # Closes the logger
  writer.close()

vocabulary size: 10001
WordLSTM(
  (dropout): Dropout(p=0.6, inplace=False)
  (emb_layer): Embedding(10002, 250)
  (lstm): LSTM(250, 250, num_layers=2, batch_first=True, dropout=0.6)
  (fc): Linear(in_features=250, out_features=10002, bias=True)
)
 bptt_len = 10 lr = 0.001 n_hidden = 250 batch_size = 30
model saved.
Epoch: 1
	 Training loss 6.13949, Training perplexity 463.82
	 Validation loss 5.73308, Validation perplexity 308.92
-----------------------------------------------------
model saved.
Epoch: 2
	 Training loss 5.79260, Training perplexity 327.86
	 Validation loss 5.53569, Validation perplexity 253.58
-----------------------------------------------------
After training:
model saved.
model saved.
	 Validation loss 5.53569, Validation perplexity 253.58
	 Test loss 5.48296, Test perplexity 240.56
-----------------------------------------------------
graph(%self.1 : __torch__.WordLSTM,
      %input.1 : Long(30, 10, strides=[10, 1], requires_grad=0, device=cuda:0),
      %27 : (Fl

In [ ]:
!tensorboard dev upload \
  --logdir \runs \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot

Please go to https://tensorboard.dev/experiment/KuKeRDFaTa2dntoswT21gQ/#scalars link to see the hyperparameter tuning results.